#  FLASK API
    ## UNDERSTAND HOW FLASK WORKS
    ## WHAT DOES THE WEB UI look like?

# Link the Finance tab to the DEPT ID tab

In [1]:
%config InlineBackend.figure_format = 'retina'

import plotly.plotly as py
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display

# Mapping data

In [2]:
df = pd.read_csv('../data/raw/CPS_School_Locations_SY1516.csv')
df['type'] = 'scattergeo'
df['locationmode'] = 'USA-states'
df['lon'] = df['Long']
df['lat'] = df['Lat']
df['name'] = df['Short_Name']

df_mapping = df

# Pull in CPS salary data

In [3]:
# Import datasets to explore
df_2017_12 = pd.read_excel('../data/raw/EmployeePositionRoster_12312017.xls', header=0)
df_2016_12 = pd.read_excel('../data/raw/EmployeePositionRoster_12312016.xls', header=0)
df_2015_12 = pd.read_excel('../data/raw/EmployeePositionRoster_12312015.xls', header=0)

# Make 2015 data have consistent naming
df_2015_12['ClsIndc'] = np.nan
col_renames = [name for name in df_2016_12.columns if name!='ClsIndc'] + ['ClsIndc']
df_2015_12.columns = col_renames
df_2015_12 = df_2015_12[df_2016_12.columns]

# Add year to DF
df_2017_12['year'] = 2017
df_2016_12['year'] = 2016
df_2015_12['year'] = 2015

df = pd.concat([df_2017_12, df_2016_12, df_2015_12])

# Check that rows were all appended
assert(df.shape[0] == df_2017_12.shape[0] + df_2016_12.shape[0] + df_2015_12.shape[0])

df.head()

,Pos #,Dept ID,Department,FTE,ClsIndc,Annual Salary,FTE Annual Salary,Annual Benefit Cost,JobCode,Job Title,Name,year
0,111427,49021,Northside Learning Center,1.0,T,75996.0,75996.0,27080.0,500629,Special Education Teacher,"McKernin, Kristen F",2017
1,111427,49021,Northside Learning Center,1.0,T,90155.0,90155.0,30232.0,500629,Special Education Teacher,"Prusaitis, Dan P",2017
2,111428,11675,Diverse Lrn Related Svc Prv CW,1.0,T,67485.0,67485.0,25185.0,984100,School Social Worker,"Gunn, Monique N",2017
3,111435,23231,Telpochcalli School,0.5,T,46236.0,46236.0,10228.0,55,Bilingual Teacher,"Ceron, Marisol",2017
4,111436,23131,Esmond School,1.0,T,95009.0,95009.0,31312.0,49,Regular Teacher,"Jeff, Lisa Y",2017


In [4]:
# format df and rename columns to prepare for pivot
temp = df.groupby(['Dept ID', 'Department', 'Job Title', 'year'])['Annual Salary', 'Annual Benefit Cost'].agg(['count','sum','mean']).reset_index()
temp.columns = [' '.join(col).strip().lower().replace(' ', '_') for col in temp.columns.values]
# display(temp.head())

# create a pivot table:
pivot = pd.pivot_table(temp,
               index=['dept_id', 'department'],
               columns='year',
               values='annual_salary_sum',
               aggfunc='sum'
              )
# display(pivot.head())

pivot['2016_delta'] = pivot[2016] - pivot[2015]
pivot['2017_delta'] = pivot[2017] - pivot[2016]


df_funding = pivot[['2016_delta','2017_delta']].reset_index()

# format df and rename columns to prepare for pivot
temp = df.groupby(['Dept ID', 'Department', 'Job Title', 'year'])['Annual Salary', 'Annual Benefit Cost'].agg(['count','sum','mean']).reset_index()
temp.columns = [' '.join(col).strip().lower().replace(' ', '_') for col in temp.columns.values]
# display(temp.head())

# create a pivot table:
pivot = pd.pivot_table(temp,
               index=['dept_id', 'department'],
               columns='year',
               values='annual_salary_count',
               aggfunc='sum'
              )
# display(pivot.head())

pivot['2016_delta_emps'] = pivot[2016] - pivot[2015]
pivot['2017_delta_emps'] = pivot[2017] - pivot[2016]

df_emps = pivot[['2016_delta_emps','2017_delta_emps']].reset_index()

df_deltas = pd.concat([df_funding, df_emps], axis=1)

In [5]:
df_link = pd.read_csv('../data/raw/Chicago_Public_Schools_-_School_Profile_Information_SY1617.csv')

In [6]:
df_link = df_link[['School_ID','Legacy_Unit_ID','Finance_ID','Long_Name']]
df_link.head()

,School_ID,Legacy_Unit_ID,Finance_ID,Long_Name
0,610158,5720,29271,Harriet E Sayre Elementary Language Academy
1,610282,7040,26301,Ronald E McNair Elementary School
2,609996,4020,23821,Charles N Holden Elementary School
3,400079,3344,66395,Acero Charter Schools - SPC Daniel Zizumbo
4,610089,5020,24621,John B Murphy Elementary School


In [7]:
df_mapping.set_index('School_ID').sort_values('Short_Name')

,Short_Name,the_geom,Address,Zip,Lat,Long,Network,Governance,Grade_Cat,Grades,Phone,GeoNetwork,COMMAREA,WARD_15,ALD_15,type,locationmode,lon,lat,name
School_ID,,,,,,,,,,,,,,,,,,,,
400010,ACE TECH HS,POINT (-87.6258490370775 41.79612150915272),5410 S State St,60609,41.796122,-87.625849,Charter,Charter,HS,"9, 10, 11, 12",1(773)548-8705,9,WASHINGTON PARK,3,Patricia R. Dowell,scattergeo,USA-states,-87.625849,41.796122,ACE TECH HS
609772,ADDAMS,POINT (-87.53297604186888 41.698738799501506),10810 S Avenue H,60617,41.698739,-87.532976,Network 13,District,ES,"PK, K, 1, 2, 3, 4, 5, 6, 7, 8",1(773)535-6210,13,EAST SIDE,10,John A. Pope,scattergeo,USA-states,-87.532976,41.698739,ADDAMS
609773,AGASSIZ,POINT (-87.65622866787353 41.93412629732926),2851 N Seminary Ave,60657,41.934126,-87.656229,Network 4,District,ES,"PK, K, 1, 2, 3, 4, 5, 6, 7, 8",1(773)534-5725,4,LAKE VIEW,44,Thomas Tunney,scattergeo,USA-states,-87.656229,41.934126,AGASSIZ
400060,AHS - PASSAGES,POINT (-87.67116041959915 41.983371435594066),1643 W Bryn Mawr Ave,60660,41.983371,-87.671160,Charter,Charter,ES,"K, 1, 2, 3, 4, 5, 6, 7, 8",1(773)433-3530,2,EDGEWATER,40,Patrick O'Connor,scattergeo,USA-states,-87.671160,41.983371,AHS - PASSAGES
610513,AIR FORCE HS,POINT (-87.63275698281277 41.828145321834505),3630 S Wells St,60609,41.828145,-87.632757,SLA,District,HS,"9, 10, 11, 12",1(773)535-1590,6,ARMOUR SQUARE,11,James A. Balcer,scattergeo,USA-states,-87.632757,41.828145,AIR FORCE HS
610212,ALBANY PARK,POINT (-87.7106087883064 41.97150444314325),4929 N Sawyer Ave,60625,41.971504,-87.710609,Network 1,District,ES,"7, 8",1(773)534-5108,1,ALBANY PARK,33,Deborah Mell,scattergeo,USA-states,-87.710609,41.971504,ALBANY PARK
609774,ALCOTT ES,POINT (-87.64641313687183 41.9298498518581),2625 N Orchard St,60614,41.929850,-87.646413,Network 4,District,ES,"PK, K, 1, 2, 3, 4, 5, 6, 7, 8",1(773)534-5460,4,LINCOLN PARK,43,Michele Smith,scattergeo,USA-states,-87.646413,41.929850,ALCOTT ES
610524,ALCOTT HS,POINT (-87.68056201292157 41.935794555495015),2957 N Hoyne Ave,60618,41.935795,-87.680562,Network 4,District,HS,"9, 10, 11, 12",1(773)534-5970,4,NORTH CENTER,32,Scott Waguespack,scattergeo,USA-states,-87.680562,41.935795,ALCOTT HS
609848,ALDRIDGE,POINT (-87.60647384722472 41.65740496963247),630 E 131st St,60827,41.657405,-87.606474,Network 13,District,ES,"PK, K, 1, 2, 3, 4, 5, 6, 7, 8",1(773)535-5614,13,RIVERDALE,9,Anthony A. Beale,scattergeo,USA-states,-87.606474,41.657405,ALDRIDGE


In [8]:
df.sort_values('Department').head()

,Pos #,Dept ID,Department,FTE,ClsIndc,Annual Salary,FTE Annual Salary,Annual Benefit Cost,JobCode,Job Title,Name,year
18531,299921,25871,A N Pritzker School,0.5,NaN,91884.19,45942.10,15308.0,49,Regular Teacher,"Schwartz, Miss Mindy Rose",2015
5480,132947,25871,A N Pritzker School,1.0,NaN,79757.01,79757.01,27917.0,49,Regular Teacher,"Bogacki, Mrs. Kathryn J",2015
26347,487268,25871,A N Pritzker School,0.5,T,40567.00,40567.00,9597.0,55,Bilingual Teacher,"Delgado, Gabriela",2016
27785,494570,25871,A N Pritzker School,1.0,T,93665.00,93665.00,31013.0,49,Regular Teacher,"Rosenwasser, Amy",2016
27950,495554,25871,A N Pritzker School,0.5,T,44158.00,44158.00,9996.0,49,Regular Teacher,"Kenyatta, Anjail J",2016


In [9]:
df_deltas.sort_values('department').head()

ValueError: Cannot sort by duplicate column department

In [10]:
columnNumbers = [x for x in range(df_deltas.reset_index().shape[1])]  # list of columns' integer indices

print(columnNumbers)
columnNumbers.remove(0)
columnNumbers.remove(1) #removing column integer index 0
columnNumbers.remove(2) 
print(columnNumbers)
df_deltas = df_deltas.reset_index().iloc[:,np.array(columnNumbers)] #return all columns except 0th column

[0, 1, 2, 3, 4, 5, 6, 7, 8]
[3, 4, 5, 6, 7, 8]


In [11]:
df_tmp = pd.merge(df_deltas, df_link, how='inner', left_on='dept_id', right_on='Finance_ID')
df_plot = pd.merge(df_tmp, df_mapping, how='inner', left_on='School_ID', right_on='School_ID')

In [12]:
df_plot['color'] = ['red' if x < 0 else 'green' for x in df_plot['2017_delta']]

In [13]:
df_plot['size'] = abs(df_plot['2017_delta'])/abs(df_plot['2017_delta']).max()*7 + 2

In [14]:
data = df_plot.loc[:,['type','locationmode','lon','lat','name', 'color', 'size']].to_dict(orient='records')
display(data)

[{'color': 'green',
  'lat': 41.811069,
  'locationmode': 'USA-states',
  'lon': -87.704199,
  'name': 'COLUMBIA EXPLORERS',
  'size': 3.686794874162158,
  'type': 'scattergeo'},
 {'color': 'green',
  'lat': 41.698739,
  'locationmode': 'USA-states',
  'lon': -87.532976,
  'name': 'ADDAMS',
  'size': 2.0838480405733946,
  'type': 'scattergeo'},
 {'color': 'green',
  'lat': 41.934126,
  'locationmode': 'USA-states',
  'lon': -87.65622900000001,
  'name': 'AGASSIZ',
  'size': 2.4999969225277128,
  'type': 'scattergeo'},
 {'color': 'green',
  'lat': 41.935795,
  'locationmode': 'USA-states',
  'lon': -87.680562,
  'name': 'ALCOTT HS',
  'size': 3.4928387219873085,
  'type': 'scattergeo'},
 {'color': 'green',
  'lat': 41.935795,
  'locationmode': 'USA-states',
  'lon': -87.680562,
  'name': 'ALCOTT HS',
  'size': nan,
  'type': 'scattergeo'},
 {'color': 'green',
  'lat': 41.92985,
  'locationmode': 'USA-states',
  'lon': -87.646413,
  'name': 'ALCOTT ES',
  'size': 3.4928387219873085,
  't

In [16]:
for i, v in enumerate(data):
    data[i]['marker'] = dict(
            size = 5,  #data[i]['size'],
            color = data[i]['color'],
            line = dict(width=0.5, color='rgb(40,40,40)'),
            sizemode = 'area'
        )
    data[i]['lat'] = pd.Series(data[i]['lat'])
    data[i]['lon'] = pd.Series(data[i]['lon'])
    del data[i]['size']
    del data[i]['color']

In [32]:
layout = dict(
        title = 'Chicago schools',
        showlegend = True,
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ), #albers usa, Mercator
            showland = True,
            landcolor = 'rgb(217, 217, 217)', 
            subunitwidth=1,
            countrywidth=1,
            subunitcolor="rgb(255, 255, 255)",
            countrycolor="rgb(255, 255, 255)",
            lonaxis = dict( range= [-70, -60] ),
            lataxis = dict( range= [45, 57] )
        )
    )

fig = dict( data=data, layout=layout )
py.iplot( fig, validate=False, filename='d3-bubble-map-populations' )

High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~andrewlai57/0 or inside your plot.ly account where it is named 'd3-bubble-map-populations'
